In [1]:
# Pre-setting
import random
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
import scipy.io

# Load MatLab m data File
xMFile = scipy.io.loadmat('x.mat')
yMFile = scipy.io.loadmat('y.mat')
xx = xMFile['xx']
yy = yMFile['yy']

In [3]:
def Log_p_eta_given_y(x, y, eta):
    
    DF = np.exp(-(x / 360.0) * (y / 100.0))
    N = len(x)
    
    t = np.unique(x)
    n = len(t)
    t_head = t[1:]
    t_tail = t[:-1]
    h = t_head - t_tail
    h = np.insert(h, 0, 0)
    
    w = np.zeros((n, 1))
    y_bar = np.zeros((n, 1))
    SSE = 0.0
    
    for j in range(n):
        w[j] = np.sum(x == t[j])
        y_bar[j] = np.sum((x == t[j]) * DF) / w[j]
        SE = (x == t[j]) * (DF - y_bar[j])
        SSE += sum(SE ** 2)
        
    Q_prime = np.zeros((n-2, n))
    T_Col_N = n - 1
    T_m = np.zeros((n-2, T_Col_N))
    
    for j in range(n-2):
        Q_prime[j, j] = (h[j + 1] + 0.0) ** (-1)
        Q_prime[j, j + 2] = (h[j + 2] + 0.0) ** (-1)
        Q_prime[j, j + 1] = -(Q_prime[j, j] + Q_prime[j, j + 2])
        
        T_m[j, j] = (h[j + 1] + h[j + 2]) / 6.0
        T_m[j, j + 1] = h[j + 2] / 6.0
        
    maskT = range(T_Col_N)
    del maskT[-2]
    T_m = T_m[:, maskT]
    T_m = T_m + T_m.T
    W = np.diag([x[0] for x in w]) # np.diag(w)
    Q = Q_prime.T
    A = Q.dot(np.linalg.inv(T_m)).dot(Q.T)
    
    
    log_p_eta = -2 * np.log(1 + eta)
    p = 2
    log_1 = ((n - p) / 2.0) * np.log(eta)
    log_3 = ((N - p) / 2.0) * np.log(y_bar.T.dot(W - W.dot(np.linalg.inv(W + eta * A)).dot(W)).dot(y_bar) + SSE)
    
    
    if eta > 1:
        log_2 = 0.5 * n * np.log(eta) + 0.5 * np.log(np.linalg.det(W / eta + A))
    else:
        log_2 = 0.5 * np.log(np.linalg.det(W + eta * A))
    
    return log_p_eta + log_1 - log_2 - log_3

In [4]:
print Log_p_eta_given_y(xx, yy, 0.0001)

print Log_p_eta_given_y(xx, yy, 0.3)

[[ 300.00236633]]
[[ 868.20796415]]


In [5]:
from scipy import integrate

# Reference from https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.integrate.quad.html
invexp = lambda eta: (Log_p_eta_given_y(xx, yy, eta))
y, err = integrate.quad(invexp, 0.01, np.inf)

C:\Anaconda2\lib\site-packages\scipy\integrate\quadpack.py:357: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)


In [1]:
print 'Integral is %f' % y
print 'Error is %f' % err

NameError: name 'y' is not defined